In [1]:
#Import Section
import carla
import math
import random
import time


In [2]:
#Load Carla Client & world
client = carla.Client('localhost',2000)
world = client.load_world('Town04')

world = client.get_world()


In [3]:
#Get the blueprint lib
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [4]:
#Set ego vehicle
ego_bp = bp_lib.find('vehicle.mercedes.coupe_2020')
ego_vehicle = world.try_spawn_actor(ego_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=0.1,z=1.5)),ego_vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [5]:
def DeleteAllVehicles():
    client.apply_batch_sync([carla.command.DestroyActor(x) for x in world.get_actors().filter('*vehicle*')])

In [6]:
def followEgoVehWindow():
    spectator = world.get_spectator()
    transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=0.1,z=1.5)),ego_vehicle.get_transform().rotation)
    spectator.set_transform(transform)

In [7]:
followEgoVehWindow()

In [8]:
#Not all the cars inside carla have headlights so we are filtering the blueprint library in order to get only the models with headlights/tailights 
def hasLights(blueprint):
    ret = False
    for attr in blueprint:
        if(attr.id=='has_lights' ):
            if(str(attr)=='ActorAttribute(id=has_lights,type=bool,value=True(const))'):
                return True
    return ret


vehicle_blueprints = [bp for bp in world.get_blueprint_library().filter('*vehicle*') if hasLights(bp)] #.filter(lambda a: hasLights(a))




In [9]:
ego_vehicle.set_autopilot(True)

In [10]:
actor_list = []
actor_list.append(ego_vehicle)
print('created %s' % ego_vehicle.type_id)

created vehicle.mercedes.coupe_2020


In [11]:
image_w = 1920
image_h = 1080

In [12]:
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_bp.set_attribute('image_size_x', str(image_w))
camera_bp.set_attribute('image_size_y', str(image_h))
camera_init_trans = carla.Transform(carla.Location(x =-0.1,z=1.5),carla.Rotation(10,0,0)) 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)
actor_list.append(camera_bp)
print('created %s' % camera_bp)


created ActorBlueprint(id=sensor.camera.rgb,tags=[sensor, camera, rgb])


In [13]:
seg_cam_bp = bp_lib.find('sensor.camera.semantic_segmentation') 
seg_cam_bp.set_attribute('image_size_x', str(image_w))
seg_cam_bp.set_attribute('image_size_y', str(image_h))
seg_cam_init_trans = carla.Transform(carla.Location(x =-0.1,z=1.5),carla.Rotation(10,0,0)) 
seg_cam = world.spawn_actor(seg_cam_bp, seg_cam_init_trans, attach_to=ego_vehicle)
actor_list.append(seg_cam_bp)
print('created %s' % seg_cam)

created Actor(id=26, type=sensor.camera.semantic_segmentation)


In [14]:


image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()


In [15]:
def rgb_callback(image,nameBuffer, imgBuffer):
    nameBuffer.append(image.frame)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) #Reshaping with alpha channel
    img[:,:,3] = 255 #Setting the alpha to 255 
    imgBuffer.append(img)

    


In [16]:
def seg_callback(image, nameBuffer, imgBuffer):
    nameBuffer.append(image.frame)
    #image.convert(carla.ColorConverter.CityScapesPalette)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) #Reshaping with alpha channel
    img[:,:,3] = 255 #Setting the alpha to 255 
    imgBuffer.append(img)



In [17]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [18]:
weather = carla.WeatherParameters(cloudiness=20.000000, precipitation=0.000000, precipitation_deposits=0.000000, wind_intensity=10.000000, sun_altitude_angle=-90.000000, fog_density=2.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000)

world.set_weather(weather)

print(world.get_weather())

WeatherParameters(cloudiness=20.000000, precipitation=0.000000, precipitation_deposits=0.000000, wind_intensity=10.000000, sun_azimuth_angle=0.000000, sun_altitude_angle=-90.000000, fog_density=2.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.000000, rayleigh_scattering_scale=0.033100, dust_storm=0.000000)


In [19]:
for vehicle in world.get_actors().filter('*vehicle*'):
    lights = carla.VehicleLightState.Position
    lights |= carla.VehicleLightState.LowBeam
    #vehicle.set_light_state(lights)
    vehicle.set_light_state(carla.VehicleLightState(lights))
    

In [20]:
lm = world.get_lightmanager()
lights = lm.get_all_lights()
lm.turn_on(lights)

In [21]:
lm = world.get_lightmanager()
lights = lm.get_all_lights(carla.LightGroup.Vehicle)
for light in lights:
    light.set_intensity(2000.0)

In [22]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

In [23]:
import numpy as np
import cv2
import time

settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.01
world.apply_settings(settings)



#rgbData = {'rgb_image': np.zeros((image_h, image_w, 4))}
rgbNameBuffer = []
rgbImgBuffer = []
camera.listen(lambda image: rgb_callback(image, rgbNameBuffer, rgbImgBuffer ))
#segData = {'seg_image': np.zeros((image_h, image_w, 4))}
segNameBuffer = []
segImgBuffer = []
seg_cam.listen(lambda image: seg_callback(image, segNameBuffer, segImgBuffer))

In [24]:

MAX_BUFFER_SIZE = 1000
counter = 0 

In [ ]:
while 1:
    try:
    
        if(counter >= MAX_BUFFER_SIZE):
            counter = 0
            for name,img in zip(rgbNameBuffer,rgbImgBuffer):
                path = '_out2/' + time.strftime("%Y%m%d-%H%M") +"-"+ str(name)+".png"
                cv2.imwrite(path, img)
            rgbNameBuffer.clear()
            rgbImgBuffer.clear()
            for name,img in zip(segNameBuffer,segImgBuffer):
                path = '_outSeg2/' + time.strftime("%Y%m%d-%H%M") +"-"+ str(name)+".png"
                cv2.imwrite(path, img)
            segNameBuffer.clear()
            segImgBuffer.clear()
        else:
            counter +=1
            world.tick()
    except: 
        print("")
